# Data Modelling for Medium Size Bikes & Cycling Accessories Organization
> This project was done under the umbrella of KPMG internship experience. I was provided data sets of an organization targeting a client who wants a feedback from us on their dataset quality and how this can be improved.

### Background
- Sprocket Central Pty Ltd, a medium size bikes & cycling accessories organisation
- needs help with its customer and transactions data
- how to analyse it to help optimise its marketing strategy effectively.

### Datasets
- New Customer List
- Customer Demographic
- Customer Addresses
- Transactions data in the past 3 months

### Task
- Build a model to predict which new customers will convert into paying customers.
- Use the RFM analysis based cluster segmentation to identify the target customers.
- Build Dashboards in Tableau/PowerBI to present your findings.